In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta, timezone
from urllib.parse import urlparse
from dateutil import parser
import unicodedata
import spacy
import numpy as np
import os

# === CONFIGURACIÓN GENERAL ===
terminos_busqueda = ["Ecopetrol", "Tarifas eléctricas", "Regasificación","Regasificadora","Soberanía energética", "Crisis del gas",
                     "Deuda del Gobierno", "Subsidios de energía","Gas Natural", "EnfraGen", "Enfragen","Afinia","EPM", "Celsia",
                     "Enel Colombia", "Cens","Compañía Energética de Occidente","Electrohuila","Cedenar","Essa","Dispac","Air-e", "Ecopetrol",
                    "Ministerio de Minas y Energía","Ministro de Minas y Energía","Comisión de Regulación de Energía y Gas","CREG",
                     "Edwin Palma", "Karen Schutt", "María del Mar Pizarro", "Juan Fernández","Ricardo Roa","Termoflores", "Termovalle",
                     "Termonorte", "Tebsa", "Termocandelaria"] 
modo = "rango"  # Opciones: "exacto" o "rango"
debug = True     # True para imprimir detalles por noticia
dias_rango = 1  # Solo si modo = "rango"
fecha_objetivo = datetime(2025, 11, 2).date()  # Solo si modo = "exacto"

medios = ["eltiempo.com", "semana.com", "elespectador.com", "larepublica.co",
    "caracol.com.co", "rcnradio.com", "wradio.com.co", "elcolombiano.com",
    "lasillavacia.com", "elheraldo.co", "bluradio.com", "cambiocolombia.com",
    "noticias.caracoltv.com", "elpais.com/america-colombia", "pulzo.com",
    "eluniversal.com.co", "publimetro.co", "elpais.com.co", "las2orillas.co",
    "minuto30.com", "kienyke.com", "lafm.com.co/politica", "portafolio.co",
    "vanguardia.com", "elmundo.com", "valoraanalitik.com", "voragine.co",
    "diariodelsur.com.co", "laopinion.co", "eldiario.com.co", "elnuevosiglo.com.co",
    "cronicadelquindio.com", "diariolalibertad.com/sitio", "lapatria.com", "elnuevodia.com.co",
    "elmeridiano.co", "infobae.com/colombia/","elperiodicodelaenergia.com/", "valoraanalitik.com",
    "businesswire.com/newsroom?language=es", "elheraldo.co","zonacero.com", "reuters.com",
    "apnews.com", "ifmnoticias.com", "canal1.com.co","ntn24.com/colombia", "90minutos.co","chicanoticias.com", "elirreverenteibague.com",
    "elcronista.co", "bloomberglinea.com/latinoamerica/colombia/"]


headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36"
}

nlp = spacy.load("es_core_news_sm")

# Funciones auxiliares
def normalizar(texto):
    if not isinstance(texto, str):
        return ""
    texto = unicodedata.normalize("NFKD", texto)
    texto = texto.encode("ascii", "ignore").decode("utf-8")
    return texto.lower()

def extraer_adjetivos(texto):
    doc = nlp(str(texto))
    return [token.text for token in doc if token.pos_ == "ADJ"]



palabras_positivas = ["éxito", "triunfó", "logro", "logros", "avance", "progreso", "mejoría", "crecimiento","baja desempleo","baja inflación",
    "decomiso de fentanilo","Suspenden Contingencia Ambiental","suspende contingencia ambiental","Suspendida la contingencia ambiental",
    "aprobación","suma otra inversión","felicidad","Guardia Nacional asegura","GN aseguran","felices","feliz","Golpe al narco",
    "decomiso","decomisan","firmeza de Sheinbaum","detienen","aseguran dosis","aseguran cargamento","aseguran artefactos","aprehenden","detenidos",
    "elementos navales descubren","presume Sheinbaum","Sheinbaum celebra","Sheinbaum inaugura","rescatan","Inaugura Sheinbaum","invierte",
    "incentivar","Entrega Clara Brugada","tendrá inversión","anuncia inversión","bienvenidas las inversiones","impulsa el desarrollo",
    "impulsa inversión","potencia minería","revoluciona la minería","Avanza la minería","minería del futuro","Más vivo",
    "impulsa la minería","amplía meta", "impulsan innovación", "transforma","innovación","auge","repunta","construcción sostenible", 
    "construcción sustentable","innovaciones","eficiencia","avanza","detonan construcción", "Obtiene reconocimiento", "fortalece",
    "señales de recuperación","crece el sector", "impulsa","bienvenidas","bienvenida","facilidades","vanguardia","inclusión",
    "inicia la construcción", "anuncian la construcción","anuncia proyectos","iniciará su construcción", "anuncia la construcción",
    "arranca construcción", "anuncia construcción","acelerará construcción", "acelera construcción", "acuerdan construcción",
    "inicia proyecto","agradece","innovadores","inicia construcción","nuevo hospital","nuevos hospitales","resalta construcción",
    "banderazo","anuncia una inversión", "iniciará construcción", "iniciará la construcción","inicio la construcción", 
    "inician construcción","optimismo","incautan","rescatan", "decomiso","decomisan","detención", "confiscan", "detenidos",
    "inversión millonaria", "beneficiará", "celebra", "golpe al peso", "Lanzan programas","forteleció el peso","fortalece le peso",
    "peso fortalecido"]

palabras_negativas = ["fracaso", "crisis", "colapso", "escándalo", "corrupción","irresponsabilidad", "problema","deterioro", "violencia", 
    "homicidio", "irrespetuosas", "peligroso", "peligrosa", "fallida","podrido", "difícil","dura", "duro", "incongruente", 
    "retrocede", "grave","traición", "pierde", "desaprobación", "desaprueba","engaña", "engaño", "sucio", "sucia", "perdería", 
    "fallido", "mal", "malo", "mala", "débil","violación","pesadilla", "desplazamiento", "polémica", "desapariciones", "cacería",
    "polémicas", "colapsó", "ineficiente", "agujero", "fallé", "desastre","vergüenza", "aumento del desempleo", "Mencho","Chapitos",
    "Mayiza","Teuchitlán","Izaguirre","exterminio","fosa clandestina","CJNG","Cártel de Sinaloa","Activan Contingencia Ambiental",
    "chantaje","chantajea","horror","mala calidad del aire","huachicol","fuga de gas","toman casetas","secuestrado","asesinados",
    "Izaguirre","contaminación","Ayotzinapa","balacera","infierno","Cartel Jalisco Nueva Generación","obesidad","sobrepeso","bombardeo",
    "sismo","tiroteo","incendio","muere","muerte","explosión","reconoce fallas","ejecutan","polvo","bloqueo","rapta","cuerpos","atentado",
    "sequía","asesinato","contra México","sanciones a México","aranceles","arancel", "terrorismo","terrorista","matanza","ejecutaron","ejecutan",
    "Descomposición","balaceras","Peso cae","narco mexicano incursiona","estafan","retroceso","cae","afectarán","afecta",
    "recesión","riesgo","desaceleración","retrasos","daños","Myanmar", "denuncian", "afectan", "ilelgal","devastación",
    "Protesta", "retrasaría","inconformidades","rechazan","clausura","desmienten construcción","suspender","frenar construcción",
    "protestan", "protestas","repudia","repudian","recortan expectativa", "Teuchitlán","Izaguirre","cárteles","cártel","homicidios",
    "Tren de Aragua","Cártel de Sinaloa","armado","violencia","baleado","fentanilo","narco","comando","asesinada","asesinado","asesinados","asesinadas",
    "asesinan","robo","robos","secuestro","secuestros","clandestino","clandestina","clandestinas","clandestinos","balean",
    "delitos","Chapitos","Mayiza","arma","Marina","violencia","Comisión Nacional de Búsqueda", "personas desaparecidas","crimen organizado",
    "Guardia Nacional","GN","narcotraficantes","disparó","dispararon","armas","aduana","ejecutan","narcorancho","asesinar","narcotraficante",
    "desaparición forzada","Los Cantaritos","desapariciones","atentado","ejecutados","homicidio","desaparecidos",
    "tiros","reclutamiento forzado", "exterminio","rifle","fusil","drogas","explosivos","clandestino","narcotráfico","muertos",
    "militares","exmilitares","marinos","tiroteo","madres buscadoras","desapariciones","raptadas","matan","asesinato","terrorismo",
    "matanza","semefo","ejecutaron","ejecutan","balaceras","capos","delincuentes","abrazos no balazos", "mencho",
    "derecho de piso","Los Alegres del Barranco","Doña Carlota","disparando","rearma","incautan", "Teresa González Murillo",
    "crimen", "familia michoacana","contrabando", "Carlota N", "sin vida","operativos", "balacera","roban","El mayo", "desaparecido","CJNG",
    "feminicidio", "aranceles recíprocos", "tira", "escasez de agua", "restos humanos", "mutilado", "linchan","linchamiento", "Cae confianza"]

def clasificar_sentimiento(titulo, adjetivos):
    titulo_norm = normalizar(titulo)
    palabras_pos = [word for word in palabras_positivas if normalizar(word) in titulo_norm]
    palabras_neg = [word for word in palabras_negativas if normalizar(word) in titulo_norm]

    if palabras_pos and not palabras_neg:
        return "Positiva"
    elif palabras_neg and not palabras_pos:
        return "Negativa"
    else:
        return "Neutral"

def palabras_clasificadoras(titulo):
    titulo_norm = normalizar(titulo)
    return ", ".join([word for word in palabras_positivas + palabras_negativas if normalizar(word) in titulo_norm])

noticias = []
ahora = datetime.now(timezone.utc)

for termino in terminos_busqueda:
    print(f"\n🔍 Buscando: {termino}")

    for medio in medios:
        dominio = urlparse("https://" + medio).netloc
        url = f"https://news.google.com/rss/search?q={termino}+site:{dominio}&hl=es-CO&gl=CO&ceid=CO:es"

        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            print(f"⚠️ Error al obtener noticias de {dominio}")
            continue

        soup = BeautifulSoup(response.content, "xml")
        items = soup.find_all("item")
        if debug:
            print(f"⚙️  {dominio} → {len(items)} noticias")

        for item in items:
            title = item.title.text.strip()
            link = item.link.text.strip()
            description = item.description.text.strip() if item.description else ""
            pub_date_str = item.pubDate.text.strip()

            try:
                pub_date = parser.parse(pub_date_str)
            except Exception:
                continue

            if modo == "exacto":
                if pub_date.date() != fecha_objetivo:
                    continue
            elif modo == "rango":
                if ahora - pub_date > timedelta(days=dias_rango):
                    continue

            contenido = f"{title} {description}".lower()
            if termino.lower() in contenido:
                if debug:
                    print(f"📰 {title} | {pub_date.date()} | {dominio}")
                noticias.append([pub_date_str, title, link, dominio, termino])
# 📁 Crear carpeta si no existe
carpeta_salida = "noticias"
os.makedirs(carpeta_salida, exist_ok=True)
# Guardar resultados
if noticias:
    df = pd.DataFrame(noticias, columns=["Fecha", "Título", "Enlace", "Fuente", "Término"])
    df["Título"] = df["Título"].str.split(" - ").str[0]
    df = df.drop_duplicates(subset=["Título", "Fuente"])
    df["Fecha"] = pd.to_datetime(df["Fecha"], errors="coerce")
    df = df[df["Fecha"].notna()]
    df["Fecha"] = df["Fecha"].dt.strftime("%d/%m/%y")
    df["Adjetivos"] = df["Título"].apply(extraer_adjetivos)
    df["Sentimiento"] = df.apply(lambda row: clasificar_sentimiento(row["Título"], row["Adjetivos"]), axis=1)
    df["Palabras Clave"] = df["Título"].apply(palabras_clasificadoras)
    df["Dominio Limpio"] = df["Fuente"].apply(lambda x: urlparse("https://" + str(x)).netloc.replace("www.", "").split("/")[0])

    sufijo_fecha = fecha_objetivo.strftime("%d-%m-%Y") if modo == "exacto" else ahora.strftime("%d-%m-%Y")
    nombre_archivo = os.path.join(
        carpeta_salida, f"noticias_enfragen_{sufijo_fecha}.csv"
)
    df.to_csv(nombre_archivo, index=False, encoding="utf-8-sig")
    print(f"\n✅ Archivo guardado como '{nombre_archivo}' con {len(df)} noticias")
else:
    print("\n❌ No se encontraron noticias que cumplan los filtros.")



🔍 Buscando: Ecopetrol


⚙️  eltiempo.com → 100 noticias


⚙️  semana.com → 100 noticias


⚙️  elespectador.com → 100 noticias


⚙️  larepublica.co → 100 noticias


...

⚙️  90minutos.co → 0 noticias


⚙️  chicanoticias.com → 0 noticias


⚙️  elirreverenteibague.com → 0 noticias


⚙️  elcronista.co → 0 noticias


⚙️  bloomberglinea.com → 0 noticias

✅ Archivo guardado como 'noticias/noticias_enfragen_01-01-2026.csv' con 11 noticias
